In [3]:
import pandas as pd

# 讀取Excel文件
file_path = 'C:/Users/WCHuang8/Desktop/學習推薦系統專案/coursera爬蟲/所有課程資訊/course_data_AI_cleaned.xlsx'
df = pd.read_excel(file_path)

# 定義一個函數來提取分級部分
def extract_level(detail):
    if isinstance(detail, str):
        return detail.split('·')[0].strip()
    return None

# 應用該函數來創建新的一列
df['course_level'] = df['course_detail'].apply(extract_level)

# 檢查新的一列
print(df[['course_detail', 'course_level']])

# 將結果保存到新的Excel文件
output_file_path = 'C:/Users/WCHuang8/Desktop/學習推薦系統專案/model/course_data_AI_with_levels.xlsx'
df.to_excel(output_file_path, index=False)

print("分級提取完成，結果已保存到新的Excel文件。")


                                         course_detail  course_level
0                     Beginner · Course · 1 - 3 Months      Beginner
1    Beginner · Professional Certificate · 3 - 6 Mo...      Beginner
2         Intermediate · Specialization · 1 - 3 Months  Intermediate
3             Beginner · Specialization · 1 - 3 Months      Beginner
4                  Intermediate · Course · 1 - 4 Weeks  Intermediate
..                                                 ...           ...
631                   Beginner · Course · 1 - 3 Months      Beginner
632                   Beginner · Course · 1 - 3 Months      Beginner
633       Intermediate · Specialization · 3 - 6 Months  Intermediate
634                   Beginner · Course · 1 - 3 Months      Beginner
635                   Beginner · Course · 1 - 3 Months      Beginner

[636 rows x 2 columns]
分級提取完成，結果已保存到新的Excel文件。


In [2]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import jieba
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

# 讀取Excel文件
file_path = 'C:/Users/WCHuang8/Desktop/學習推薦系統專案/coursera爬蟲/所有課程資訊/course_data_AI_cleaned.xlsx'
df = pd.read_excel(file_path)

# 刪除包含缺漏值（NaN）的行
df = df.dropna()

# 提取課程名稱和介紹文本
df['text'] = df['course_title'] + ' ' + df['skill_gain'] + ' ' + df['course_detail']

# 分詞
df['words'] = df['text'].apply(lambda x: ' '.join(jieba.cut(x)))

# 使用TF-IDF提取關鍵詞
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df['words'])
feature_names = vectorizer.get_feature_names_out()

# 提取每個課程的關鍵詞
def get_top_keywords(tfidf_row, feature_names, top_n=10):
    sorted_nzs = tfidf_row.nonzero()[1][tfidf_row.data.argsort()[::-1]]
    top_keywords = [feature_names[i] for i in sorted_nzs[:top_n]]
    return top_keywords

df['keywords'] = df.apply(lambda row: get_top_keywords(tfidf_matrix[row.name], feature_names), axis=1)

# 假設我們有一列 'course_level' 代表課程的等級（高/中/低）
# 這裡我們隨機生成一些示例數據
import numpy as np
np.random.seed(0)
df['course_level'] = np.random.choice(['高', '中', '低'], size=len(df))

# 將 'course_level' 映射到數值標籤
label_mapping = {'低': 0, '中': 1, '高': 2}
df['label'] = df['course_level'].map(label_mapping)

# 切分數據集
X = tfidf_matrix
y = df['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 訓練模型
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# 預測
y_pred = clf.predict(X_test)

# 評估模型
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:")
print(classification_report(y_test, y_pred))

# 輸出包含關鍵詞和等級的課程資料
df['predicted_level'] = clf.predict(tfidf_matrix)
df['predicted_course_level'] = df['predicted_level'].map({0: '低', 1: '中', 2: '高'})
print(df[['course_title', 'keywords', 'course_level', 'predicted_course_level']])

# 如果需要將結果保存為新的Excel文件
output_file_path = 'C:/Users/WCHuang8/Desktop/學習推薦系統專案/model/course_data_AI_with_predictions.xlsx'
df.to_excel(output_file_path, index=False)

print("課程分級完成，結果已保存到新的Excel文件。")


Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\WCHuang8\AppData\Local\Temp\jieba.cache
Loading model cost 0.787 seconds.
Prefix dict has been built successfully.


Accuracy: 0.390625
Classification Report:
              precision    recall  f1-score   support

           0       0.26      0.26      0.26        35
           1       0.43      0.52      0.47        44
           2       0.46      0.37      0.41        49

    accuracy                           0.39       128
   macro avg       0.38      0.38      0.38       128
weighted avg       0.39      0.39      0.39       128

                                          course_title  \
0                                 Google AI Essentials   
1                                     IBM AI Developer   
2          Introduction to Generative AI Learning Path   
3                      IBM AI Foundations for Business   
4             Generative AI with Large Language Models   
..                                                 ...   
631            Data Analysis with Spreadsheets and SQL   
632  Project Management Foundations, Initiation, an...   
633                  International Business Essentials 

coursera 80% 20%

3. 劃分數據集
將數據集劃分為訓練集（80%）和測試集（20%），以便我們可以評估模型的性能。

4. 訓練模型
使用訓練數據來訓練隨機森林分類器。隨機森林通過以下步驟進行建模：

構建多棵決策樹：隨機森林通過對原始數據進行多次有放回的隨機抽樣（稱為Bootstrap樣本）來構建多棵決策樹。每棵樹都是在不同的訓練樣本上訓練的。

隨機特徵選擇：在每棵決策樹的每個節點上，隨機森林會隨機選擇一個特徵子集，並在這個特徵子集上選擇最佳分割點，從而增加樹之間的多樣性。

In [31]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
import jieba

# 讀取Excel文件
file_path = 'C:/Users/WCHuang8/Desktop/學習推薦系統專案/model/course_data_AI_with_levels.xlsx'
df = pd.read_excel(file_path)

# 確保數據中沒有缺失值
df = df.dropna()

# 提取文本特徵
df['text'] = df['course_title'] + ' ' + df['skill_gain'] + ' ' + df['course_detail']

# 分詞
df['words'] = df['text'].apply(lambda x: ' '.join(jieba.cut(x)))

# 映射課程級別到數值標籤
label_mapping = {'Beginner': 0, 'Intermediate': 1, 'Advanced': 2}  # 假設有三個級別
df['label'] = df['course_level'].map(label_mapping)

# 檢查是否存在NaN值並刪除
df = df.dropna(subset=['label'])

# 使用TF-IDF提取文本特徵
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df['words'])

# 檢查數據集大小是否一致
print(f"TF-IDF Matrix shape: {tfidf_matrix.shape}")
print(f"Labels shape: {df['label'].shape}")

# 劃分數據集，80%訓練數據和20%測試數據
X_train, X_test, y_train, y_test = train_test_split(tfidf_matrix, df['label'], test_size=0.2, random_state=42)

# 檢查劃分後的數據集大小
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")

# 訓練隨機森林模型
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# 預測
y_pred = clf.predict(X_test)

# 評估模型
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:")
print(classification_report(y_test, y_pred))

# 確保索引的一致性
print(f"y_test index: {list(y_test.index)}")

# 獲取測試數據的索引
test_indices = y_test.index

# 如果需要將測試結果保存到Excel文件
df_test = df.loc[test_indices].copy()  # 使用 .loc 確保索引一致性
df_test['predicted_label'] = y_pred
df_test['predicted_course_level'] = df_test['predicted_label'].map({0: 'Beginner', 1: 'Intermediate', 2: 'Advanced'})

test_output_file_path = 'C:/Users/WCHuang8/Desktop/學習推薦系統專案/model/course_data_test_with_predictions.xlsx'
df_test.to_excel(test_output_file_path, index=False)

print("模型訓練和預測完成，結果已保存到新的Excel文件。")

TF-IDF Matrix shape: (600, 903)
Labels shape: (600,)
X_train shape: (480, 903)
X_test shape: (120, 903)
y_train shape: (480,)
y_test shape: (120,)
Accuracy: 0.9916666666666667
Classification Report:
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00        85
         1.0       0.97      1.00      0.98        30
         2.0       1.00      0.80      0.89         5

    accuracy                           0.99       120
   macro avg       0.99      0.93      0.96       120
weighted avg       0.99      0.99      0.99       120

y_test index: [112, 434, 600, 78, 187, 292, 10, 492, 79, 357, 56, 120, 111, 624, 379, 242, 30, 219, 190, 87, 2, 623, 564, 632, 378, 569, 73, 139, 589, 454, 631, 167, 83, 265, 198, 473, 136, 542, 458, 283, 374, 29, 532, 605, 24, 188, 497, 106, 64, 515, 253, 256, 103, 588, 627, 365, 216, 169, 141, 201, 77, 180, 337, 565, 579, 71, 596, 625, 82, 324, 312, 6, 439, 384, 445, 566, 320, 246, 593, 448, 294, 442, 634, 371, 74,

Udemy vs Coursera

In [21]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
import jieba

# 讀取之前的資料集並訓練模型
train_file_path = 'C:/Users/WCHuang8/Desktop/學習推薦系統專案/model/course_data_AI_with_levels.xlsx'
df_train = pd.read_excel(train_file_path)

# 確保數據中沒有缺失值
df_train = df_train.dropna()

# 提取文本特徵
df_train['text'] = df_train['course_title'] + ' ' + df_train['skill_gain']

# 分詞
df_train['words'] = df_train['text'].apply(lambda x: ' '.join(jieba.cut(x)))

# 映射課程級別到數值標籤
label_mapping = {'Beginner': 0, 'Intermediate': 1, 'Advanced': 2}
df_train['label'] = df_train['course_level'].map(label_mapping)

# 檢查是否存在NaN值並刪除
df_train = df_train.dropna(subset=['label'])

# 使用TF-IDF提取文本特徵
vectorizer = TfidfVectorizer()
tfidf_matrix_train = vectorizer.fit_transform(df_train['words'])

# 劃分數據集，80%訓練數據和20%測試數據
X_train, X_test, y_train, y_test = train_test_split(tfidf_matrix_train, df_train['label'], test_size=0.2, random_state=42)

# 訓練隨機森林模型
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# 在測試集上進行預測
y_pred = clf.predict(X_test)

# 評估模型
print("Accuracy on test data:", accuracy_score(y_test, y_pred))
print("Classification Report on test data:")
print(classification_report(y_test, y_pred))

# 讀取新的資料集
new_file_path = 'C:/Users/WCHuang8/Desktop/學習推薦系統專案/model/UdemyCourses.xlsx'
df_new = pd.read_excel(new_file_path)

# 確保數據中沒有缺失值
df_new = df_new.dropna()

# 提取文本特徵
df_new['text'] = df_new['course_title'] + ' ' + df_new['skill_gain']

# 分詞
df_new['words'] = df_new['text'].apply(lambda x: ' '.join(jieba.cut(x)))

# 使用之前的TF-IDF向量化器轉換新資料集
tfidf_matrix_new = vectorizer.transform(df_new['words'])

# 使用訓練好的模型進行預測
predicted_labels = clf.predict(tfidf_matrix_new)

# 將預測結果映射回課程級別
df_new['predicted_label'] = predicted_labels
df_new['predicted_course_level'] = df_new['predicted_label'].map({0: 'Beginner', 1: 'Intermediate', 2: 'Advanced'})

# 保存預測結果到新的Excel文件
output_file_path = 'C:/Users/WCHuang8/Desktop/學習推薦系統專案/model/UdemyCourses_with_predictions.xlsx'
df_new.to_excel(output_file_path, index=False)

print("預測完成，結果已保存到新的Excel文件。")


Accuracy on test data: 0.7333333333333333
Classification Report on test data:
              precision    recall  f1-score   support

         0.0       0.76      0.94      0.84        85
         1.0       0.53      0.27      0.36        30
         2.0       0.00      0.00      0.00         5

    accuracy                           0.73       120
   macro avg       0.43      0.40      0.40       120
weighted avg       0.67      0.73      0.69       120



c:\Users\WCHuang8\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\WCHuang8\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\WCHuang8\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(av

預測完成，結果已保存到新的Excel文件。


Udemy

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
import jieba

# 讀取Excel文件
file_path = 'C:/Users/WCHuang8/Desktop/學習推薦系統專案/model/UdemyCourses.xlsx'
df = pd.read_excel(file_path)

# 確保數據中沒有缺失值
df = df.dropna()

# 提取文本特徵
df['text'] = df['course_title'] + ' ' + df['skill_gain'] 

# 分詞
df['words'] = df['text'].apply(lambda x: ' '.join(jieba.cut(x)))

# 映射課程級別到數值標籤
label_mapping = {'Beginner': 0, 'Intermediate': 1, 'Advanced': 2}
df['label'] = df['course_level'].map(label_mapping)

# 檢查是否存在NaN值並刪除
df = df.dropna(subset=['label'])

# 使用TF-IDF提取文本特徵
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df['words'])

# 檢查數據集大小是否一致
print(f"TF-IDF Matrix shape: {tfidf_matrix.shape}")
print(f"Labels shape: {df['label'].shape}")

# 劃分數據集，80%訓練數據和20%測試數據
X_train, X_test, y_train, y_test = train_test_split(tfidf_matrix, df['label'], test_size=0.2, random_state=42)

# 檢查劃分後的數據集大小
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")

# 訓練隨機森林模型
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# 預測
y_pred = clf.predict(X_test)

# 評估模型
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:")
print(classification_report(y_test, y_pred))

# 確保索引的一致性
print(f"y_test index: {list(y_test.index)}")

# 獲取測試數據的索引
test_indices = y_test.index

# 如果需要將測試結果保存到Excel文件
df_test = df.loc[test_indices].copy()
df_test['predicted_label'] = y_pred
df_test['predicted_course_level'] = df_test['predicted_label'].map({0: 'Beginner', 1: 'Intermediate', 2: 'Advanced'})

test_output_file_path = 'C:/Users/WCHuang8/Desktop/學習推薦系統專案/model/course_data_test_with_predictions_2.xlsx'
df_test.to_excel(test_output_file_path, index=False)

print("模型訓練和預測完成，結果已保存到新的Excel文件。")

In [4]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
import jieba

# 讀取Excel文件
file_path = 'C:/Users/WCHuang8/Desktop/學習推薦系統專案/model/UdemyCourses.xlsx'
df = pd.read_excel(file_path)

# 確保數據中沒有缺失值
df = df.dropna()

# 提取文本特徵
df['text'] = df['course_title'] + ' ' + df['skill_gain'] 

# 分詞
df['words'] = df['text'].apply(lambda x: ' '.join(jieba.cut(x)))

# 映射課程級別到數值標籤
label_mapping = {'Beginner': 0, 'Intermediate': 1, 'Advanced': 2}
df['label'] = df['course_level'].map(label_mapping)

# 檢查是否存在NaN值並刪除
df = df.dropna(subset=['label'])

# 使用TF-IDF提取文本特徵
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df['words'])

# 檢查數據集大小是否一致
print(f"TF-IDF Matrix shape: {tfidf_matrix.shape}")
print(f"Labels shape: {df['label'].shape}")

# 劃分數據集，80%訓練數據和20%測試數據
X_train, X_test, y_train, y_test = train_test_split(tfidf_matrix, df['label'], test_size=0.2, random_state=42)

# 檢查劃分後的數據集大小
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")

# 使用GridSearchCV調整隨機森林模型超參數
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'max_features': ['auto', 'sqrt', 'log2'],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

clf = RandomForestClassifier(random_state=42)
grid_search = GridSearchCV(estimator=clf, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

print("Best Parameters:", grid_search.best_params_)
clf = grid_search.best_estimator_

# 預測
y_pred = clf.predict(X_test)

# 評估模型
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:")
print(classification_report(y_test, y_pred))

# 確保索引的一致性
print(f"y_test index: {list(y_test.index)}")

# 獲取測試數據的索引
test_indices = y_test.index

# 如果需要將測試結果保存到Excel文件
df_test = df.loc[test_indices].copy()
df_test['predicted_label'] = y_pred
df_test['predicted_course_level'] = df_test['predicted_label'].map({0: 'Beginner', 1: 'Intermediate', 2: 'Advanced'})

test_output_file_path = 'C:/Users/WCHuang8/Desktop/學習推薦系統專案/model/course_data_test_with_predictions_2.xlsx'
df_test.to_excel(test_output_file_path, index=False)

print("模型訓練和預測完成，結果已保存到新的Excel文件。")


TF-IDF Matrix shape: (6055, 9946)
Labels shape: (6055,)
X_train shape: (4844, 9946)
X_test shape: (1211, 9946)
y_train shape: (4844,)
y_test shape: (1211,)
Fitting 5 folds for each of 324 candidates, totalling 1620 fits


c:\Users\WCHuang8\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
540 fits failed out of a total of 1620.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
244 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\WCHuang8\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\WCHuang8\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py", line 1466, in wrapper
    estimator._validate_params()
  File "c:\Users\WCHuang8\AppData\Local\Programs\Python\Python312\Lib\site-packages

Best Parameters: {'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 300}
Accuracy: 0.7555739058629232
Classification Report:
              precision    recall  f1-score   support

         0.0       0.76      0.99      0.86       887
         1.0       0.70      0.12      0.20       286
         2.0       0.45      0.13      0.20        38

    accuracy                           0.76      1211
   macro avg       0.64      0.41      0.42      1211
weighted avg       0.74      0.76      0.68      1211

y_test index: [5589, 730, 2543, 1550, 2468, 4579, 254, 2884, 2729, 4829, 1851, 3205, 2885, 3005, 5239, 5076, 1896, 2444, 6012, 2646, 5142, 4382, 534, 5352, 4685, 2435, 2010, 3394, 5747, 319, 5314, 707, 5971, 2376, 753, 1956, 5420, 2082, 2341, 5966, 4244, 3116, 5052, 23, 1259, 465, 2822, 2786, 3053, 79, 6040, 1618, 323, 3619, 5397, 1556, 5961, 4547, 5211, 5374, 894, 4904, 4815, 1158, 6142, 3453, 2934, 2230, 4150, 1531, 3391, 2250, 221,

其他方式

In [3]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
import jieba
from xgboost import XGBClassifier

# 讀取Excel文件
file_path = 'C:/Users/WCHuang8/Desktop/學習推薦系統專案/model/UdemyCourses.xlsx'
df = pd.read_excel(file_path)

# 確保數據中沒有缺失值
df = df.dropna()

# 提取文本特徵
df['text'] = df['course_title'] + ' ' + df['skill_gain'] 

# 分詞
df['words'] = df['text'].apply(lambda x: ' '.join(jieba.cut(x)))

# 映射課程級別到數值標籤
label_mapping = {'Beginner': 0, 'Intermediate': 1, 'Advanced': 2}
df['label'] = df['course_level'].map(label_mapping)

# 檢查是否存在NaN值並刪除
df = df.dropna(subset=['label'])

# 使用TF-IDF提取文本特徵
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df['words'])

# 檢查數據集大小是否一致
print(f"TF-IDF Matrix shape: {tfidf_matrix.shape}")
print(f"Labels shape: {df['label'].shape}")

# 劃分數據集，80%訓練數據和20%測試數據
X_train, X_test, y_train, y_test = train_test_split(tfidf_matrix, df['label'], test_size=0.2, random_state=42)

# 檢查劃分後的數據集大小
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")

# 訓練XGBoost模型
clf = XGBClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# 預測
y_pred = clf.predict(X_test)

# 評估模型
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:")
print(classification_report(y_test, y_pred))

# 確保索引的一致性
print(f"y_test index: {list(y_test.index)}")

# 獲取測試數據的索引
test_indices = y_test.index

# 如果需要將測試結果保存到Excel文件
df_test = df.loc[test_indices].copy()
df_test['predicted_label'] = y_pred
df_test['predicted_course_level'] = df_test['predicted_label'].map({0: 'Beginner', 1: 'Intermediate', 2: 'Advanced'})

test_output_file_path = 'C:/Users/WCHuang8/Desktop/學習推薦系統專案/model/course_data_test_with_predictions_2.xlsx'
df_test.to_excel(test_output_file_path, index=False)

print("模型訓練和預測完成，結果已保存到新的Excel文件。")


Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\WCHuang8\AppData\Local\Temp\jieba.cache
Loading model cost 0.828 seconds.
Prefix dict has been built successfully.


TF-IDF Matrix shape: (6055, 9946)
Labels shape: (6055,)
X_train shape: (4844, 9946)
X_test shape: (1211, 9946)
y_train shape: (4844,)
y_test shape: (1211,)
Accuracy: 0.7522708505367465
Classification Report:
              precision    recall  f1-score   support

         0.0       0.78      0.95      0.86       887
         1.0       0.55      0.23      0.32       286
         2.0       0.33      0.11      0.16        38

    accuracy                           0.75      1211
   macro avg       0.55      0.43      0.45      1211
weighted avg       0.71      0.75      0.71      1211

y_test index: [5589, 730, 2543, 1550, 2468, 4579, 254, 2884, 2729, 4829, 1851, 3205, 2885, 3005, 5239, 5076, 1896, 2444, 6012, 2646, 5142, 4382, 534, 5352, 4685, 2435, 2010, 3394, 5747, 319, 5314, 707, 5971, 2376, 753, 1956, 5420, 2082, 2341, 5966, 4244, 3116, 5052, 23, 1259, 465, 2822, 2786, 3053, 79, 6040, 1618, 323, 3619, 5397, 1556, 5961, 4547, 5211, 5374, 894, 4904, 4815, 1158, 6142, 3453, 2934, 2230, 4

In [4]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
import jieba

# 讀取Excel文件
file_path = 'C:/Users/WCHuang8/Desktop/學習推薦系統專案/model/UdemyCourses.xlsx'
df = pd.read_excel(file_path)

# 確保數據中沒有缺失值
df = df.dropna()

# 提取文本特徵
df['text'] = df['course_title'] + ' ' + df['skill_gain'] 

# 分詞
df['words'] = df['text'].apply(lambda x: ' '.join(jieba.cut(x)))

# 映射課程級別到數值標籤
label_mapping = {'Beginner': 0, 'Intermediate': 1, 'Advanced': 2}
df['label'] = df['course_level'].map(label_mapping)

# 檢查是否存在NaN值並刪除
df = df.dropna(subset=['label'])

# 使用TF-IDF提取文本特徵
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df['words'])

# 檢查數據集大小是否一致
print(f"TF-IDF Matrix shape: {tfidf_matrix.shape}")
print(f"Labels shape: {df['label'].shape}")

# 劃分數據集，80%訓練數據和20%測試數據
X_train, X_test, y_train, y_test = train_test_split(tfidf_matrix, df['label'], test_size=0.2, random_state=42)

# 檢查劃分後的數據集大小
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")

# 訓練SVM模型
clf = SVC(kernel='linear', random_state=42)
clf.fit(X_train, y_train)

# 預測
y_pred = clf.predict(X_test)

# 評估模型
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:")
print(classification_report(y_test, y_pred))

# 確保索引的一致性
print(f"y_test index: {list(y_test.index)}")

# 獲取測試數據的索引
test_indices = y_test.index

# 如果需要將測試結果保存到Excel文件
df_test = df.loc[test_indices].copy()
df_test['predicted_label'] = y_pred
df_test['predicted_course_level'] = df_test['predicted_label'].map({0: 'Beginner', 1: 'Intermediate', 2: 'Advanced'})

test_output_file_path = 'C:/Users/WCHuang8/Desktop/學習推薦系統專案/model/course_data_test_with_predictions_2.xlsx'
df_test.to_excel(test_output_file_path, index=False)

print("模型訓練和預測完成，結果已保存到新的Excel文件。")


TF-IDF Matrix shape: (6055, 9946)
Labels shape: (6055,)
X_train shape: (4844, 9946)
X_test shape: (1211, 9946)
y_train shape: (4844,)
y_test shape: (1211,)
Accuracy: 0.7555739058629232
Classification Report:
              precision    recall  f1-score   support

         0.0       0.77      0.98      0.86       887
         1.0       0.61      0.16      0.25       286
         2.0       0.43      0.08      0.13        38

    accuracy                           0.76      1211
   macro avg       0.60      0.40      0.41      1211
weighted avg       0.72      0.76      0.69      1211

y_test index: [5589, 730, 2543, 1550, 2468, 4579, 254, 2884, 2729, 4829, 1851, 3205, 2885, 3005, 5239, 5076, 1896, 2444, 6012, 2646, 5142, 4382, 534, 5352, 4685, 2435, 2010, 3394, 5747, 319, 5314, 707, 5971, 2376, 753, 1956, 5420, 2082, 2341, 5966, 4244, 3116, 5052, 23, 1259, 465, 2822, 2786, 3053, 79, 6040, 1618, 323, 3619, 5397, 1556, 5961, 4547, 5211, 5374, 894, 4904, 4815, 1158, 6142, 3453, 2934, 2230, 4

In [7]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report, accuracy_score
import jieba
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, Embedding, Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

# 讀取Excel文件
file_path = 'C:/Users/WCHuang8/Desktop/學習推薦系統專案/model/UdemyCourses.xlsx'
df = pd.read_excel(file_path)

# 確保數據中沒有缺失值
df = df.dropna()

# 提取文本特徵
df['text'] = df['course_title'] + ' ' + df['skill_gain'] 

# 分詞
df['words'] = df['text'].apply(lambda x: ' '.join(jieba.cut(x)))

# 映射課程級別到數值標籤
label_mapping = {'Beginner': 0, 'Intermediate': 1, 'Advanced': 2}
df['label'] = df['course_level'].map(label_mapping)

# 檢查是否存在NaN值並刪除
df = df.dropna(subset=['label'])

# 將文本轉換為序列
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['words'])
sequences = tokenizer.texts_to_sequences(df['words'])

# 進行填充，使所有序列具有相同長度
max_len = 100
X = pad_sequences(sequences, maxlen=max_len)

# 將標籤進行One-Hot編碼
lb = LabelBinarizer()
y = lb.fit_transform(df['label'])

# 劃分數據集，80%訓練數據和20%測試數據
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 構建CNN模型
vocab_size = len(tokenizer.word_index) + 1  # 加1是因為索引從1開始

model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=128, input_length=max_len))
model.add(Conv1D(filters=128, kernel_size=5, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))  # 3個類別

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 訓練模型
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)

# 預測
y_pred_prob = model.predict(X_test)
y_pred = np.argmax(y_pred_prob, axis=1)
y_test_labels = np.argmax(y_test, axis=1)

# 評估模型
print("Classification Report:")
print(classification_report(y_test_labels, y_pred, target_names=lb.classes_.astype(str)))
print("Accuracy:", accuracy_score(y_test_labels, y_pred))

# 如果需要將測試結果保存到Excel文件
df_test = df.iloc[y_test_labels.index].copy()
df_test['predicted_label'] = y_pred
df_test['predicted_course_level'] = df_test['predicted_label'].map({0: 'Beginner', 1: 'Intermediate', 2: 'Advanced'})

test_output_file_path = 'C:/Users/WCHuang8/Desktop/學習推薦系統專案/model/course_data_test_with_predictions_cnn.xlsx'
df_test.to_excel(test_output_file_path, index=False)

print("模型訓練和預測完成，結果已保存到新的Excel文件。")


Epoch 1/50


c:\Users\WCHuang8\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


122/122 ━━━━━━━━━━━━━━━━━━━━ 5s 28ms/step - accuracy: 0.7361 - loss: 0.7101 - val_accuracy: 0.7234 - val_loss: 0.6396
Epoch 2/50
122/122 ━━━━━━━━━━━━━━━━━━━━ 3s 27ms/step - accuracy: 0.7688 - loss: 0.5051 - val_accuracy: 0.7183 - val_loss: 0.7496
Epoch 3/50
122/122 ━━━━━━━━━━━━━━━━━━━━ 3s 27ms/step - accuracy: 0.8857 - loss: 0.3112 - val_accuracy: 0.6894 - val_loss: 0.8438
Epoch 4/50
122/122 ━━━━━━━━━━━━━━━━━━━━ 3s 27ms/step - accuracy: 0.9467 - loss: 0.1517 - val_accuracy: 0.6852 - val_loss: 1.2277
Epoch 5/50
122/122 ━━━━━━━━━━━━━━━━━━━━ 3s 27ms/step - accuracy: 0.9733 - loss: 0.0784 - val_accuracy: 0.6987 - val_loss: 1.5148
Epoch 6/50
122/122 ━━━━━━━━━━━━━━━━━━━━ 3s 27ms/step - accuracy: 0.9870 - loss: 0.0450 - val_accuracy: 0.7049 - val_loss: 1.8664
Epoch 7/50
122/122 ━━━━━━━━━━━━━━━━━━━━ 3s 27ms/step - accuracy: 0.9947 - loss: 0.0227 - val_accuracy: 0.6894 - val_loss: 2.0840
Epoch 8/50
122/122 ━━━━━━━━━━━━━━━━━━━━ 3s 27ms/step - accuracy: 0.9972 - loss: 0.0141 - val_accuracy: 0.702

AttributeError: 'numpy.ndarray' object has no attribute 'index'

In [19]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report, accuracy_score
import jieba
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, Embedding, Dropout, GlobalMaxPooling1D
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# 讀取Excel文件
file_path = 'C:/Users/WCHuang8/Desktop/學習推薦系統專案/model/UdemyCourses.xlsx'
df = pd.read_excel(file_path)

# 確保數據中沒有缺失值
df = df.dropna()

# 提取文本特徵
df['text'] = df['course_title'] + ' ' + df['skill_gain'] 

# 分詞
df['words'] = df['text'].apply(lambda x: ' '.join(jieba.cut(x)))

# 映射課程級別到數值標籤
label_mapping = {'Beginner': 0, 'Intermediate': 1, 'Advanced': 2}
df['label'] = df['course_level'].map(label_mapping)

# 檢查是否存在NaN值並刪除
df = df.dropna(subset=['label'])

# 將文本轉換為序列
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['words'])
sequences = tokenizer.texts_to_sequences(df['words'])

# 進行填充，使所有序列具有相同長度
max_len = 100
X = pad_sequences(sequences, maxlen=max_len)

# 將標籤進行One-Hot編碼
lb = LabelBinarizer()
y = lb.fit_transform(df['label'])

# 劃分數據集，80%訓練數據和20%測試數據
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 構建CNN模型
vocab_size = len(tokenizer.word_index) + 1  # 加1是因為索引從1開始

model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=128, input_length=max_len))
model.add(Conv1D(filters=128, kernel_size=5, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.5))
model.add(Conv1D(filters=64, kernel_size=5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))  # 3個類別

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 訓練模型
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=0.0001)

model.fit(X_train, y_train, epochs=15, batch_size=32, validation_split=0.2, callbacks=[early_stopping, reduce_lr])

# 預測
y_pred_prob = model.predict(X_test)
y_pred = np.argmax(y_pred_prob, axis=1)
y_test_labels = np.argmax(y_test, axis=1)

# 評估模型
print("Classification Report:")
print(classification_report(y_test_labels, y_pred, target_names=lb.classes_.astype(str)))
print("Accuracy:", accuracy_score(y_test_labels, y_pred))

# 如果需要將測試結果保存到Excel文件
df_test = df.iloc[y_test_labels.index].copy()
df_test['predicted_label'] = y_pred
df_test['predicted_course_level'] = df_test['predicted_label'].map({0: 'Beginner', 1: 'Intermediate', 2: 'Advanced'})

test_output_file_path = 'C:/Users/WCHuang8/Desktop/學習推薦系統專案/model/course_data_test_with_predictions_cnn.xlsx'
df_test.to_excel(test_output_file_path, index=False)

print("模型訓練和預測完成，結果已保存到新的Excel文件。")


Epoch 1/15


c:\Users\WCHuang8\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


122/122 ━━━━━━━━━━━━━━━━━━━━ 5s 25ms/step - accuracy: 0.7392 - loss: 0.7391 - val_accuracy: 0.7234 - val_loss: 0.6458 - learning_rate: 0.0010
Epoch 2/15
122/122 ━━━━━━━━━━━━━━━━━━━━ 3s 23ms/step - accuracy: 0.7338 - loss: 0.5803 - val_accuracy: 0.7193 - val_loss: 0.6491 - learning_rate: 0.0010
Epoch 3/15
122/122 ━━━━━━━━━━━━━━━━━━━━ 3s 23ms/step - accuracy: 0.8155 - loss: 0.4302 - val_accuracy: 0.6801 - val_loss: 0.8197 - learning_rate: 0.0010
Epoch 4/15
122/122 ━━━━━━━━━━━━━━━━━━━━ 3s 23ms/step - accuracy: 0.8894 - loss: 0.2828 - val_accuracy: 0.6760 - val_loss: 0.9776 - learning_rate: 2.0000e-04
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
Classification Report:
              precision    recall  f1-score   support

         0.0       0.73      1.00      0.85       887
         1.0       0.00      0.00      0.00       286
         2.0       0.00      0.00      0.00        38

    accuracy                           0.73      1211
   macro avg       0.24      0.33      0.28      1211
weighte

c:\Users\WCHuang8\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\WCHuang8\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\WCHuang8\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(av

AttributeError: 'numpy.ndarray' object has no attribute 'index'

In [23]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report, accuracy_score
import jieba
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# 日誌記錄
import logging
logging.basicConfig(level=logging.INFO)

# 讀取Excel文件
file_path = 'C:/Users/WCHuang8/Desktop/學習推薦系統專案/model/UdemyCourses.xlsx'
df = pd.read_excel(file_path)

# 確保數據中沒有缺失值
df = df.dropna()

# 提取文本特徵
df['text'] = df['course_title'] + ' ' + df['skill_gain']

# 分詞
df['words'] = df['text'].apply(lambda x: ' '.join(jieba.cut(x)))

# 映射課程級別到數值標籤
label_mapping = {'Beginner': 0, 'Intermediate': 1, 'Advanced': 2}
df['label'] = df['course_level'].map(label_mapping)

# 檢查是否存在NaN值並刪除
df = df.dropna(subset=['label'])

# 日誌記錄：顯示數據集的大小
logging.info(f"數據集大小: {df.shape}")

# 將文本轉換為序列
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['words'])
sequences = tokenizer.texts_to_sequences(df['words'])

# 進行填充，使所有序列具有相同長度
max_len = 100
X = pad_sequences(sequences, maxlen=max_len)

# 將標籤進行One-Hot編碼
lb = LabelBinarizer()
y = lb.fit_transform(df['label'])

# 劃分數據集，80%訓練數據和20%測試數據
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 日誌記錄：顯示訓練集和測試集的大小
logging.info(f"訓練集大小: {X_train.shape}, 測試集大小: {X_test.shape}")

try:
    # 構建LSTM模型
    vocab_size = len(tokenizer.word_index) + 1  # 加1是因為索引從1開始

    model = Sequential()
    model.add(Embedding(input_dim=vocab_size, output_dim=128, input_length=max_len))
    model.add(LSTM(64, return_sequences=True))  # 減少LSTM單元數
    model.add(Dropout(0.5))
    model.add(LSTM(32))  # 減少LSTM單元數
    model.add(Dense(64, activation='relu'))  # 減少Dense單元數
    model.add(Dropout(0.5))
    model.add(Dense(3, activation='softmax'))  # 3個類別

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # 訓練模型
    early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=0.0001)

    model.fit(X_train, y_train, epochs=15, batch_size=32, validation_split=0.2, callbacks=[early_stopping, reduce_lr])

    # 預測
    y_pred_prob = model.predict(X_test)
    y_pred = np.argmax(y_pred_prob, axis=1)
    y_test_labels = np.argmax(y_test, axis=1)

    # 評估模型
    print("Classification Report:")
    print(classification_report(y_test_labels, y_pred, target_names=lb.classes_.astype(str)))
    print("Accuracy:", accuracy_score(y_test_labels, y_pred))

    # 如果需要將測試結果保存到Excel文件
    df_test = df.iloc[X_test.index].copy()  # 使用X_test的索引
    df_test['predicted_label'] = y_pred
    df_test['predicted_course_level'] = df_test['predicted_label'].map({0: 'Beginner', 1: 'Intermediate', 2: 'Advanced'})

    test_output_file_path = 'C:/Users/WCHuang8/Desktop/學習推薦系統專案/model/course_data_test_with_predictions_lstm.xlsx'
    df_test.to_excel(test_output_file_path, index=False)

    print("模型訓練和預測完成，結果已保存到新的Excel文件。")
except Exception as e:
    logging.error(f"訓練過程中發生錯誤: {e}")


INFO:root:數據集大小: (6055, 10)
INFO:root:訓練集大小: (4844, 100), 測試集大小: (1211, 100)


Epoch 1/15


c:\Users\WCHuang8\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


122/122 ━━━━━━━━━━━━━━━━━━━━ 12s 65ms/step - accuracy: 0.7055 - loss: 0.8067 - val_accuracy: 0.7234 - val_loss: 0.7041 - learning_rate: 0.0010
Epoch 2/15
122/122 ━━━━━━━━━━━━━━━━━━━━ 7s 60ms/step - accuracy: 0.7486 - loss: 0.6890 - val_accuracy: 0.7183 - val_loss: 0.6478 - learning_rate: 0.0010
Epoch 3/15
122/122 ━━━━━━━━━━━━━━━━━━━━ 9s 71ms/step - accuracy: 0.8195 - loss: 0.4838 - val_accuracy: 0.7183 - val_loss: 0.7257 - learning_rate: 0.0010
Epoch 4/15
122/122 ━━━━━━━━━━━━━━━━━━━━ 9s 73ms/step - accuracy: 0.8812 - loss: 0.3364 - val_accuracy: 0.6894 - val_loss: 0.8694 - learning_rate: 0.0010
Epoch 5/15
122/122 ━━━━━━━━━━━━━━━━━━━━ 10s 80ms/step - accuracy: 0.9364 - loss: 0.2079 - val_accuracy: 0.6945 - val_loss: 1.0523 - learning_rate: 2.0000e-04
38/38 ━━━━━━━━━━━━━━━━━━━━ 2s 31ms/step


c:\Users\WCHuang8\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\WCHuang8\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\WCHuang8\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(av

Classification Report:
              precision    recall  f1-score   support

         0.0       0.79      0.94      0.86       887
         1.0       0.54      0.29      0.38       286
         2.0       0.00      0.00      0.00        38

    accuracy                           0.76      1211
   macro avg       0.44      0.41      0.41      1211
weighted avg       0.70      0.76      0.72      1211

Accuracy: 0.7572254335260116


助手

In [18]:
import pandas as pd
from difflib import get_close_matches

# 讀取本地文件
file_path = 'C:/Users/WCHuang8/Desktop/學習推薦系統專案/model/Course_Title_and_Level.csv'
user_classification_file_path = 'C:/Users/WCHuang8/Desktop/學習推薦系統專案/model/user_classification_results.xlsx'

# 加載數據
course_data = pd.read_csv(file_path)
user_classification_data = pd.read_excel(user_classification_file_path)

# 提取所需的列
course_data = course_data[['course_title', 'course_level']]
user_classification_data = user_classification_data[['course_title', 'course_level']]

# 定義一個函數來找到每個課程標題的最接近匹配
def find_closest_match(title, title_list):
    matches = get_close_matches(title, title_list, n=1, cutoff=0.6)
    return matches[0] if matches else None

# 應用該函數來找到原始數據中最接近的課程標題
original_titles = set(course_data['course_title'])
user_classification_data['matched_title'] = user_classification_data['course_title'].apply(lambda x: find_closest_match(x, original_titles))

# 合併數據集並比較課程等級
merged_data = pd.merge(course_data, user_classification_data, left_on='course_title', right_on='matched_title', suffixes=('_original', '_classified'))
merged_data['is_correct'] = merged_data['course_level_original'] == merged_data['course_level_classified']

# 計算準確率
accuracy = merged_data['is_correct'].mean()

print(f"準確率: {accuracy * 100:.2f}%")


準確率: 36.00%
